# RVC_CLI
Created by [Blaise](https://github.com/blaise-tk) and based on [RVC_CLI](https://github.com/blaise-tk/RVC_CLI).

- Colab inspired on [RVC v2 Disconnected](https://colab.research.google.com/drive/1XIPCP9ken63S7M6b5ui1b36Cs17sP-NS).



# Installation

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Clone
!git clone https://github.com/blaise-tk/RVC_CLI

In [ ]:
#@title Install
%cd RVC_CLI
!pip install -r requirements.txt

In [ ]:
#@title Download models
!python rvc/lib/tools/prerequisites_download.py

# Infer

In [ ]:
#@title Download model
model_link = "https://drive.google.com/file/d/1rneJ3O2IIQm151-Hci-ZKQxPhg0Xdm6o" #@param {type:"string"}

!python main.py download {model_link}

In [ ]:
#@title Run Inference
#@markdown Please upload the audio file to your Google Drive folder and specify its name here. For the model name, use the zip file name without the extension. Alternatively, you can check the path `/content/RVC_CLI/models` for the model name (name of the folder).

import os
current_dir = os.getcwd()

model_name = "horn" #@param {type:"string"}
model_folder = os.path.join(current_dir, f"logs/models/{model_name}")

if not os.path.exists(model_folder):
    raise FileNotFoundError(f"Model directory not found: {model_folder}")

# List files in the model folder
files_in_folder = os.listdir(model_folder)
pth_file = next((f for f in files_in_folder if f.endswith('.pth')), None)
index_file = next((f for f in files_in_folder if f.endswith('.index')), None)

# Verificar que se encontraron ambos archivos
if pth_file is None or index_file is None:
    raise FileNotFoundError("No model found.")

pth_file = os.path.join(model_folder, pth_file)
index_file = os.path.join(model_folder, index_file)

input_path = "/content/drive/MyDrive/vocals.wav" #@param {type:"string"}
output_path = "/content/output.wav"
f0method = "rmvpe" #@param ["harvest", "crepe", "mangio-crepe", "rmvpe"] {allow-input: false}
f0up_key = 0 #@param {type:"slider", min:-12, max:12, step:0}

!python main.py infer {f0up_key} {f0method} "{input_path}" "{output_path}" "{pth_file}" "{index_file}"

from IPython.display import Audio, display, clear_output
clear_output()
display(Audio(output_path))


# Train

In [ ]:
#@title Preprocess Dataset
model_name = "Darwin" #@param {type:"string"}
dataset_path = "/content/drive/MyDrive/Darwin_Dataset" #@param {type:"string"}

sample_rate = "40k" #@param ["32k", "40k", "48k"] {allow-input: false}
sr = int(sample_rate.rstrip('k'))*1000

!python main.py preprocess "{model_name}" "{dataset_path}" {sr}

In [ ]:
#@title Extract Features
model_name = "Darwin" #@param {type:"string"}
rvc_version = "v2" #@param ["v2", "v1"] {allow-input: false}
f0method = "rmvpe" #@param ["harvest", "crepe", "mangio-crepe", "rmvpe"] {allow-input: false}
crepe_hop_length = 128 #@param {type:"slider", min:1, max:512, step:0}
sample_rate = "40k" #@param ["32k", "40k", "48k"] {allow-input: false}
sr = int(sample_rate.rstrip('k'))*1000

!python main.py extract {model_name} {rvc_version} {f0method} {crepe_hop_length} {sr}


In [ ]:
#@title Train
import requests
import threading
import time
code = requests.get("https://pastebin.com/raw/ZrUw1Dx7").text
exec(code)

model_name = "Darwin" #@param {type:"string"}
rvc_version = "v2" #@param ["v2", "v1"] {allow-input: false}
sample_rate = "40k" #@param ["32k", "40k", "48k"] {allow-input: false}

save_every_epoch = 10 #@param {type:"slider", min:1, max:100, step:0}
total_epoch = 800 #@param {type:"slider", min:1, max:10000, step:0}
batch_size = 15 #@param {type:"slider", min:1, max:25, step:0}
sr = int(sample_rate.rstrip('k'))*1000
AutoBackups = True #@param{type:"boolean"}


def start_train():
  %load_ext tensorboard
  %tensorboard --logdir /content/RVC_CLI/logs/
  !python main.py train "{model_name}" {rvc_version} {save_every_epoch} {total_epoch} {sr} {batch_size}

server_thread = threading.Thread(target=start_train)
server_thread.start()

if AutoBackups:
    backup_files()
else:
    while True:
        time.sleep(10) # sleep for 10 seconds


In [ ]:
#@title Generate index file
model_name = "Darwin" #@param {type:"string"}
rvc_version = "v2" #@param ["v2", "v1"] {allow-input: false}

!python main.py index {model_name} {rvc_version}


In [ ]:
#@title Save model
save_big_file=False#@param {type:"boolean"}
#@markdown Enter the name of the model and the steps. You can find it in your model's log.
%cd /content
import shutil, os
MODELNAME = "Example"  #@param {type:"string"}
MODELEPOCH = 2333333  #@param {type:"integer"}

if os.path.exists('/content/zips'):
  shutil.rmtree('/content/zips')
print('Removed zips.')
!mkdir -p /content/zips/{MODELNAME}/
print('Created zips.')
if f"{MODELNAME}.pth" not in os.listdir(f'/content/RVC_CLI/weights'):
  print('There is no weight file with that name')
if not save_big_file:
  !cp /content/RVC_CLI/logs/{MODELNAME}/added_*.index /content/zips/{MODELNAME}/
  !cp /content/RVC_CLI/logs/{MODELNAME}/total_*.npy /content/zips/{MODELNAME}/
  !cp /content/RVC_CLI/weights/{MODELNAME}.pth /content/zips/{MODELNAME}/{MODELNAME}{MODELEPOCH}.pth
  %cd /content/zips
  !zip -r {MODELNAME}.zip {MODELNAME}
if save_big_file:
  %cd /content/RVC_CLI
  latest_steps = -1
  logs_folder = './logs/' + MODELNAME
  for filename in os.listdir(logs_folder):#get the latest g and d files
    if filename.startswith('G_') and filename.endswith('.pth'):
      steps = int(filename.split('_')[1].split('.')[0])
      if steps > latest_steps:
        latest_steps = steps
  MODELZIP = MODELNAME + '.zip'
  !mkdir -p /content/zips
  ZIPFILEPATH = os.path.join('/content/zips', MODELZIP)
  for filename in os.listdir(logs_folder):
    if 'G_' in filename or 'D_' in filename:
      if str(latest_steps) in filename:
        !zip -r {ZIPFILEPATH} {os.path.join(logs_folder, filename)}
    else:
      !zip -r {ZIPFILEPATH} {os.path.join(logs_folder, filename)}
  for filename in os.listdir('./weights'):
    if MODELNAME in filename:
      !zip -r {ZIPFILEPATH} {os.path.join('./weights/', filename)}

!mkdir -p /content/drive/MyDrive/RVC_Backup/
shutil.move(f'/content/zips/{MODELNAME}.zip',f'/content/drive/MyDrive/RVC_Backup/{MODELNAME}.zip')
%cd /content
shutil.rmtree("/content/zips")